In [ ]:
#create table records(id int primary key auto_increment,name varchar(30),pass varchar(30),email varchar(30),age varchar(5))

import socket
import mysql.connector
from _thread import *
mydb=mysql.connector.connect(host="localhost",user="root",passwd="india@123",database="client_server")
mycursor=mydb.cursor()


s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
host = socket.gethostname()
#host = '127.0.0.1'
port = 12345
NumberOfThreads=0

try:
    s.bind((host, port))
except socket.error as e:
    print(str(e))

print('Server Listening...')
print("Waiting for Client to connect")
s.listen(5)


def multiclient(conn):
    c=1
    while (c):
        option = conn.recv(1024).decode()
        #print(option)
        if (option == "0"):
            c=0
            conn.close()
            
        elif option == "1":
            username = conn.recv(1024).decode()
            password = conn.recv(1024).decode()
            login = validateLogin(username, password)
            conn.sendall(login.encode())
            if (login =="1"):
                d=1
                while (d):
                    g = conn.recv(1024).decode()
                    if (g == "3"):
                        email = conn.recv(1024).decode()
                        age = conn.recv(1024).decode()
                        update = updateInfo(username,password,email,age)
                        conn.sendall(update.encode())
                    elif (g == "4"):
                        d=0
                        delete = deleteAccount(username,password)
                        conn.sendall(delete.encode())
                    elif (g == "5"):
                        retrieve = retrieveInfo(username,password)
                        conn.sendall(retrieve.encode())
                    elif (g == "6"):
                        response = conn.recv(1024).decode()
                        Logout = logout(response)
                        conn.sendall(Logout.encode())
                        if(Logout == "1"):
                            d=0
                    else:
                        return "Invalid choice"

        elif option == "2":
            username = conn.recv(1024).decode()
            password = conn.recv(1024).decode()
            email = conn.recv(1024).decode()
            age = conn.recv(1024).decode()
            print("passing")
            create = createacc(username,password,email,age)
            print("passed")
            conn.sendall(create.encode())
            
        else:
            print("Invalid")

def createacc(username,password,email,age):
    p=[]
    mycursor.execute("select name,pass from records")
    for i in mycursor:
        p.append(i)
    if (username,password) in p:
        return "Username already exists!!"
    mycursor.execute("insert into records(name,pass,email,age) values(%s,%s,%s,%s)",(username,password,email,age))
    mydb.commit()
    return "Your Account has been created"


def validateLogin(username,password):
    p=[]
    mycursor.execute("select name,pass from records")
    for i in mycursor:
        p.append(i)
    if (username,password) in p:
        print("1")
        return "1"
    else:
        print("2")
        return "2"
    
def updateInfo(username,password,email,age):
    mycursor.execute("update records set email=%s,age=%s where name=%s and pass=%s",(email,age,username,password))
    mydb.commit()
    return "Your age & gender is updated"
    
    
def deleteAccount(username,password):
    mycursor.execute("delete from records where name=%s and pass=%s",(username,password))
    mydb.commit()
    return "Your account is deleted!"


def retrieveInfo(username,password):
    mycursor.execute("select * from records where name=%s and pass=%s",(username,password))
    p=[]
    for i in mycursor:
        p=i
    mydb.commit()
    h=("username= "+ p[1] + " , password= " + p[2] + " ,email= " + p[3] +", age= "+ p[4])
    return h


def logout(response):
    if response == ("yes") or response == ("Yes") or response == ("YES"):
        return "1"
    elif response == ("no") or response == ("No") or response == ("NO"):
        return "0"
    else:
        return "2"

    
while True:
    conn, addr = s.accept()
    print("Server is connected to :"+addr[0]+":"+str(addr[1]))
    start_new_thread(multiclient, (conn,))
    NumberOfThreads+=1
    print("Thread number of new client is: "+str(NumberOfThreads))
s.close()

Server Listening...
Waiting for Client to connect
Server is connected to :192.168.1.8:53196
Thread number of new client is: 1
